# Set up the environment

In [1]:
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
import seaborn
import os
import pickle
import json
from scipy.stats import spearmanr

# Define a function to create a filename

In [2]:
def getFileName(name, n_samples, model_name, layer_name):
    return name \
        + "_{}_".format(n_samples) \
        + "_{}_".format(model_name) \
        + "_{}".format(layer_name)  \
        + ".npy"   

# Define a function to create the upper triangular of Input RDMs

In [3]:
def get_upper_triangular(rdm):
    num_conditions = rdm.shape[0] #num samples
    return rdm[np.triu_indices(num_conditions,1)] #take all above the main diagonal (excluding it), returns flattened version

# Load the model(s) and layers

In [10]:
multiple_models = 0 #comparing within a model or between models; 1 = between, 0 = within
trained_on_1 = 'places'
trained_on_2 = 'places'
tested_on = 'places'

#load the np file containing the shape of the activations
ROOT_PATH = '/mnt/raid/ni/agnessa/RSA/'
layers_path = '/mnt/raid/ni/agnessa/RSA/'

if trained_on_1 == 'imagenet':
    path_data_1 = 'Objects/'
    if tested_on == 'imagenet':
        NR_OF_SAMPLES = 10000
    elif tested_on == 'places':
        NR_OF_SAMPLES = 10220

elif trained_on_1 == 'places':
    if tested_on == 'imagenet':
        NR_OF_SAMPLES = 10000
        path_data_1 = 'Scenes/ImageNet/'
    elif tested_on == 'places':
        NR_OF_SAMPLES = 10220
        path_data_1 = 'Scenes/Places365/'

if trained_on_2 == 'imagenet':
    path_data_2 = 'Objects/'
    if tested_on == 'imagenet':
        NR_OF_SAMPLES = 10000
    elif tested_on == 'places':
        NR_OF_SAMPLES = 10220

elif trained_on_2 == 'places':
    if tested_on == 'imagenet':
        NR_OF_SAMPLES = 10000
        path_data_2 = 'Scenes/ImageNet/'
    elif tested_on == 'places':
        NR_OF_SAMPLES = 10220
        path_data_2 = 'Scenes/Places365/'
    
arch_1 = 'alexnet'

#for resnets, use the selected layers file
if 'res' in arch_1:
    json_file_layers=os.path.join(layers_path + 'layer_names/',arch_1 + '_selected_layers.json')
else: 
    json_file_layers=os.path.join(layers_path + 'layer_names/',arch_1 + '.json')
    
with open(json_file_layers, "r") as fd:
    selected_layers_1 = json.load(fd)

#get the name of the model and of the layers
num_layers_1 = len(selected_layers_1) 
print('Model: ', arch_1, ', Number of layers: ',num_layers_1)
model_begin_1 = 0 #index of the first layer of the desired model
model_name_1 = arch_1
layer_names_1 = []

if multiple_models == 1:
    arch_2 = 'alexnet'
    
    #for resnets, use the selected layers file
    if arch_2 == 'resnet34' or arch_2 == 'resnext':
        json_file_layers=os.path.join(layers_path + 'layer_names/', 'resnets_selected_layers.json')
    elif arch_2 == 'resnet18' or arch_2 == 'resnet50':
        json_file_layers=os.path.join(layers_path + 'layer_names/',arch_2 + '_selected_layers.json')
    else: 
        json_file_layers=os.path.join(layers_path + 'layer_names/',arch_2 + '.json')
    
    with open(json_file_layers, "r") as fd:
        selected_layers_2 = json.load(fd)
        
    #get the name of the model and of the layers
    num_layers_2 = len(selected_layers_2) 
    print('Model: ', arch_2, ', Number of layers: ',num_layers_2)
    model_begin_2 = 0
    model_name_2 = arch_2
    layer_names_2 = []

#append the layers from the first model
for i in range(num_layers_1):
    layer_names_1.append(selected_layers_1[model_begin_1+i].get('layer'))   
    
if multiple_models == 1:
    for j in range(num_layers_2):  #append the layers from the second model
        layer_names_2.append(selected_layers_2[model_begin_2+j].get('layer'))
else:
    layer_names_2 = layer_names_1
if multiple_models == 1:
    print('Comparing', model_name_1, ', layers', layer_names_1, 'and', model_name_2, ', layers', layer_names_2)
else:
    print('Comparing model', model_name_1, ', layers', layer_names_1)


Model:  alexnet , Number of layers:  21
Comparing model alexnet , layers ['features', 'features.0', 'features.1', 'features.2', 'features.3', 'features.4', 'features.5', 'features.6', 'features.7', 'features.8', 'features.9', 'features.10', 'features.11', 'features.12', 'avgpool', 'classifier.0', 'classifier.1', 'classifier.2', 'classifier.3', 'classifier.4', 'classifier.5']


# Create model RDMs by correlating between Input RDMs from different layers and models

In [ ]:
size_rdm_1 = np.array(layer_names_1).shape[0]   
size_rdm_2 = np.array(layer_names_2).shape[0]
RSA_matrix = np.ones((size_rdm_1,size_rdm_2)) #num layers x num layers
RSA_matrix[:] = np.nan

if multiple_models == 1:
    model_name = model_name_1 + '_' + model_name_2
else:
    model_name = model_name_2 = model_name_1  
    layer_names_2 = layer_names_1
    
#1. get upper triangulars
#2. calculate the correlation distance (1-Spearman's coefficient) between the upper triangulars
#3. repeat for all pairs of layers (and models)

for layer_i in layer_names_1:

    ## load RDMs ##
    RDM_PATH_i = os.path.join(ROOT_PATH + path_data_1 + 'Input_RDM/', \
                              getFileName('Input_RDM_', NR_OF_SAMPLES, model_name_1, layer_i))
    Input_RDM_i = np.load(RDM_PATH_i)
    print(RDM_PATH_i)
    ## get upper triangulars, without the 0 diagonal
    print('Getting the upper triangular of ->', layer_i)
    ut_rdm_i = get_upper_triangular(Input_RDM_i)
   
    for layer_j in layer_names_2:
        
        RDM_PATH_j = os.path.join(ROOT_PATH + path_data_2 + 'Input_RDM/',\
                                  getFileName('Input_RDM_', NR_OF_SAMPLES, model_name_2, layer_j))                
        Input_RDM_j = np.load(RDM_PATH_j)
        print(RDM_PATH_j)
        print('Getting the upper triangular of ->', layer_j)
        ut_rdm_j = get_upper_triangular(Input_RDM_j)
            
        # Spearman correlation
        print('Calculating the correlation distance between ->', layer_i, 'and', layer_j)
        RSA_i_j = 1-spearmanr(ut_rdm_i,ut_rdm_j)[0]
        print('Finished the correlation distance between ->', layer_i, 'and', layer_j)
        print(RSA_i_j)
        # Save into a matrix 
        print('Saving the correlation distance between ->', layer_i, 'and', layer_j)
        RSA_matrix[np.where(np.array(layer_names_1)==layer_i)[0][0],\
                   np.where(np.array(layer_names_2)==layer_j)[0][0]] = RSA_i_j

# save model RDM 
if multiple_models == 1:
    path = os.path.join(ROOT_PATH + path_data_1 + 'Model_RDM/', getFileName('Model_RDM_cross_task_', NR_OF_SAMPLES, model_name, 'all'))
else:
    path = os.path.join(ROOT_PATH + path_data_1 + 'Model_RDM/', getFileName('Model_RDM_', NR_OF_SAMPLES, model_name, 'all'))

np.save(path,RSA_matrix)

        

/mnt/raid/ni/agnessa/RSA/Scenes/Places365/Input_RDM/Input_RDM__10220__alexnet__features.npy
Getting the upper triangular of -> features
/mnt/raid/ni/agnessa/RSA/Scenes/Places365/Input_RDM/Input_RDM__10220__alexnet__features.npy
Getting the upper triangular of -> features
Calculating the correlation distance between -> features and features
Finished the correlation distance between -> features and features
0.0
Saving the correlation distance between -> features and features
/mnt/raid/ni/agnessa/RSA/Scenes/Places365/Input_RDM/Input_RDM__10220__alexnet__features.0.npy
Getting the upper triangular of -> features.0
Calculating the correlation distance between -> features and features.0
Finished the correlation distance between -> features and features.0
0.629618302425161
Saving the correlation distance between -> features and features.0
/mnt/raid/ni/agnessa/RSA/Scenes/Places365/Input_RDM/Input_RDM__10220__alexnet__features.1.npy
Getting the upper triangular of -> features.1
Calculating the 

/mnt/raid/ni/agnessa/RSA/Scenes/Places365/Input_RDM/Input_RDM__10220__alexnet__features.0.npy
Getting the upper triangular of -> features.0
Calculating the correlation distance between -> features.0 and features.0
Finished the correlation distance between -> features.0 and features.0
1.1102230246251565e-16
Saving the correlation distance between -> features.0 and features.0
/mnt/raid/ni/agnessa/RSA/Scenes/Places365/Input_RDM/Input_RDM__10220__alexnet__features.1.npy
Getting the upper triangular of -> features.1
Calculating the correlation distance between -> features.0 and features.1
Finished the correlation distance between -> features.0 and features.1
1.1102230246251565e-16
Saving the correlation distance between -> features.0 and features.1
/mnt/raid/ni/agnessa/RSA/Scenes/Places365/Input_RDM/Input_RDM__10220__alexnet__features.2.npy
Getting the upper triangular of -> features.2
Calculating the correlation distance between -> features.0 and features.2
Finished the correlation distanc

Finished the correlation distance between -> features.1 and features.0
1.1102230246251565e-16
Saving the correlation distance between -> features.1 and features.0
/mnt/raid/ni/agnessa/RSA/Scenes/Places365/Input_RDM/Input_RDM__10220__alexnet__features.1.npy
Getting the upper triangular of -> features.1
Calculating the correlation distance between -> features.1 and features.1
Finished the correlation distance between -> features.1 and features.1
1.1102230246251565e-16
Saving the correlation distance between -> features.1 and features.1
/mnt/raid/ni/agnessa/RSA/Scenes/Places365/Input_RDM/Input_RDM__10220__alexnet__features.2.npy
Getting the upper triangular of -> features.2
Calculating the correlation distance between -> features.1 and features.2
Finished the correlation distance between -> features.1 and features.2
0.48771436524074585
Saving the correlation distance between -> features.1 and features.2
/mnt/raid/ni/agnessa/RSA/Scenes/Places365/Input_RDM/Input_RDM__10220__alexnet__feature

Calculating the correlation distance between -> features.2 and features.1
Finished the correlation distance between -> features.2 and features.1
0.48771436524074585
Saving the correlation distance between -> features.2 and features.1
/mnt/raid/ni/agnessa/RSA/Scenes/Places365/Input_RDM/Input_RDM__10220__alexnet__features.2.npy
Getting the upper triangular of -> features.2
Calculating the correlation distance between -> features.2 and features.2
Finished the correlation distance between -> features.2 and features.2
0.0
Saving the correlation distance between -> features.2 and features.2
/mnt/raid/ni/agnessa/RSA/Scenes/Places365/Input_RDM/Input_RDM__10220__alexnet__features.3.npy
Getting the upper triangular of -> features.3
Calculating the correlation distance between -> features.2 and features.3
Finished the correlation distance between -> features.2 and features.3
0.2798420893967871
Saving the correlation distance between -> features.2 and features.3
/mnt/raid/ni/agnessa/RSA/Scenes/Pla

/mnt/raid/ni/agnessa/RSA/Scenes/Places365/Input_RDM/Input_RDM__10220__alexnet__features.2.npy
Getting the upper triangular of -> features.2
Calculating the correlation distance between -> features.3 and features.2
Finished the correlation distance between -> features.3 and features.2
0.27984208939678723
Saving the correlation distance between -> features.3 and features.2
/mnt/raid/ni/agnessa/RSA/Scenes/Places365/Input_RDM/Input_RDM__10220__alexnet__features.3.npy
Getting the upper triangular of -> features.3
Calculating the correlation distance between -> features.3 and features.3
Finished the correlation distance between -> features.3 and features.3
0.0
Saving the correlation distance between -> features.3 and features.3
/mnt/raid/ni/agnessa/RSA/Scenes/Places365/Input_RDM/Input_RDM__10220__alexnet__features.4.npy
Getting the upper triangular of -> features.4
Calculating the correlation distance between -> features.3 and features.4
Finished the correlation distance between -> features.

Finished the correlation distance between -> features.4 and features.2
0.27984208939678723
Saving the correlation distance between -> features.4 and features.2
/mnt/raid/ni/agnessa/RSA/Scenes/Places365/Input_RDM/Input_RDM__10220__alexnet__features.3.npy
Getting the upper triangular of -> features.3
Calculating the correlation distance between -> features.4 and features.3
Finished the correlation distance between -> features.4 and features.3
0.0
Saving the correlation distance between -> features.4 and features.3
/mnt/raid/ni/agnessa/RSA/Scenes/Places365/Input_RDM/Input_RDM__10220__alexnet__features.4.npy
Getting the upper triangular of -> features.4
Calculating the correlation distance between -> features.4 and features.4
Finished the correlation distance between -> features.4 and features.4
0.0
Saving the correlation distance between -> features.4 and features.4
/mnt/raid/ni/agnessa/RSA/Scenes/Places365/Input_RDM/Input_RDM__10220__alexnet__features.5.npy
Getting the upper triangular o

Calculating the correlation distance between -> features.5 and features.3
Finished the correlation distance between -> features.5 and features.3
0.13749339529051063
Saving the correlation distance between -> features.5 and features.3
/mnt/raid/ni/agnessa/RSA/Scenes/Places365/Input_RDM/Input_RDM__10220__alexnet__features.4.npy
Getting the upper triangular of -> features.4
Calculating the correlation distance between -> features.5 and features.4
Finished the correlation distance between -> features.5 and features.4
0.13749339529051063
Saving the correlation distance between -> features.5 and features.4
/mnt/raid/ni/agnessa/RSA/Scenes/Places365/Input_RDM/Input_RDM__10220__alexnet__features.5.npy
Getting the upper triangular of -> features.5
Calculating the correlation distance between -> features.5 and features.5
Finished the correlation distance between -> features.5 and features.5
1.1102230246251565e-16
Saving the correlation distance between -> features.5 and features.5
/mnt/raid/ni/ag

/mnt/raid/ni/agnessa/RSA/Scenes/Places365/Input_RDM/Input_RDM__10220__alexnet__features.4.npy
Getting the upper triangular of -> features.4
Calculating the correlation distance between -> features.6 and features.4
Finished the correlation distance between -> features.6 and features.4
0.15607862595645394
Saving the correlation distance between -> features.6 and features.4
/mnt/raid/ni/agnessa/RSA/Scenes/Places365/Input_RDM/Input_RDM__10220__alexnet__features.5.npy
Getting the upper triangular of -> features.5
Calculating the correlation distance between -> features.6 and features.5
Finished the correlation distance between -> features.6 and features.5
0.1380072268497904
Saving the correlation distance between -> features.6 and features.5
/mnt/raid/ni/agnessa/RSA/Scenes/Places365/Input_RDM/Input_RDM__10220__alexnet__features.6.npy
Getting the upper triangular of -> features.6
Calculating the correlation distance between -> features.6 and features.6
Finished the correlation distance betwe

Finished the correlation distance between -> features.7 and features.4
0.15607862595645394
Saving the correlation distance between -> features.7 and features.4
/mnt/raid/ni/agnessa/RSA/Scenes/Places365/Input_RDM/Input_RDM__10220__alexnet__features.5.npy
Getting the upper triangular of -> features.5
Calculating the correlation distance between -> features.7 and features.5
Finished the correlation distance between -> features.7 and features.5
0.1380072268497904
Saving the correlation distance between -> features.7 and features.5
/mnt/raid/ni/agnessa/RSA/Scenes/Places365/Input_RDM/Input_RDM__10220__alexnet__features.6.npy
Getting the upper triangular of -> features.6
Calculating the correlation distance between -> features.7 and features.6
Finished the correlation distance between -> features.7 and features.6
1.1102230246251565e-16
Saving the correlation distance between -> features.7 and features.6
/mnt/raid/ni/agnessa/RSA/Scenes/Places365/Input_RDM/Input_RDM__10220__alexnet__features.7.